In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
display(HTML("<style>.input_area pre {font-family: Consolas; font-size: 15pt; line-height: 140%;}</style>"))
display(HTML("<style>.output_area pre {font-family: Consolas; font-size: 15pt; line-height: 140%;}</style>"))

# 제출 방법
1. 파일 이름에서 '학번'을 자신의 학번으로, '이름'을 자신의 이름으로 고치시오.


# 실습1

http://dowellcomputer.com/member/memberLoginForm.jsp 에 로그인하고 사용자 정보에서 email 주소를 추출하여 출력한다.

- 아이디: dsdsds
- 비밀번호: dsdsds


***결과***
```
Action URL: http://dowellcomputer.com/member/memberLoginAction.jsp
Cookies: {'JSESSIONID': '2DAC157995E6FF56897B9B8DA75B60FB'}
Email: dsdsds@dsdsds.com
```

In [15]:
import json
import requests as rq
from urllib.parse import urljoin
from bs4 import BeautifulSoup

login_page_url = "http://dowellcomputer.com/member/memberLoginForm.jsp"
test_url = "http://dowellcomputer.com/member/memberUpdateForm.jsp?ID=dsdsds"

'memberID' # ID input 태그 

    # action URL 출력
    
action_url = urljoin( login_page_url , './memberLoginAction.jsp' )  # action_url 
print('Action URL:', action_url)
    
login_info = {
    'memberID':'dsdsds',
    'memberPassword' : 'dsdsds'
}
    
# Cookies 출력
    
with rq.Session() as s:
        
    login_res = s.post( action_url , data = login_info) # post 
    login_soup = BeautifulSoup( login_res.text , 'lxml')
        
    print('Cookies:', dict(s.cookies) )
    # Email 정보 출력 
    res = s.get(test_url)
    soup = BeautifulSoup( res.text , 'lxml')
    
    email = soup.select_one('input[name=memberEmail]')
    print('Email:', email)
    
    pass






Action URL: http://dowellcomputer.com/member/memberLoginAction.jsp
Cookies: {'JSESSIONID': '6B1EB7EF5E3F22BFA7A4C74F74926389'}
Email: <input name="memberEmail" type="text" value="dsdsds@dsdsds.com"/>


# 실습2

https://www.wishket.com/ 에 로그인하고 프로젝트 건 수 크롤링 하기

https://www.wishket.com/partners/manage/interest/ 여기에서 관심 프로젝트의 제목과 가격을 출력한다.

아이디: crawler070

패스워드: 123123123!

***결과***
```
옵션 합성 자동매매 프로그램 개발: 2,000,000원
웹 프런트엔드 매물 선택보기 필터 기능 구현: 1,000,000원
쿠팡 특정제품 아이템 위너 및 2순위 제품 가격 추적: 2,000,000원
```

In [1]:
import sys
import io
from bs4 import BeautifulSoup
import requests as rq
from urllib.parse import urljoin
from fake_useragent import UserAgent

ua = UserAgent()


def get_wishket_csrftoken(s, login_url):
    login_res = s.get(login_url)
    login_soup = BeautifulSoup(login_res.text, 'lxml')
    csrftoken = login_soup.select_one('input[name=csrfmiddlewaretoken]').get('value')
    return csrftoken

def wishket_login(s):
    login_url = 'https://www.wishket.com/accounts/login/'
    action_url = urljoin(login_url, '/accounts/login/')
    
    csrftoken = get_wishket_csrftoken(s, login_url)
    login_info = {
        'identification':'crawler070',
        'password':'123123123!',
        'csrfmiddlewaretoken':csrftoken
    }
    
    header = {
        'User-Agent':str(ua.chrome),
        'Referer':login_url,
    }
    
    action_res = s.post(action_url,data=login_info, headers=header)


with rq.Session() as s:
    wishket_login(s)
    
    interest_url = 'https://www.wishket.com/partners/manage/interest/'
    res = s.get(interest_url)
    soup = BeautifulSoup(res.text,'lxml')
    project_list = soup.select(".interest-card")

    for i in project_list:
        title = i.select_one('.card-title').get_text()
        pay = i.select_one('.card-expectation-money-content').get_text()
        print(f"{title}: {pay}")

옵션 합성 자동매매 프로그램 개발: 2,000,000원
웹 프런트엔드 매물 선택보기 필터 기능 구현: 1,000,000원
쿠팡 특정제품 아이템 위너 및 2순위 제품 가격 추적: 2,000,000원
